# Assignment

###In this assignment, we will design the drive system for a transport belt. The drive will be powered by a geared electromotor and a chain transmission. Here are the initial specifications:###

Speed of the motor: $n_3 = 1500 \: rpm$

Team specific requirements:
- Speed of chain: $v_1 = 15 \: km/h$
- Sprocket diameter: $d_1 = 450 \: mm$
- Power needed to move the carts:  $P_1 = 4.4 \: kW$
-----

In [12]:
# import statements
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt

import rm_python_lib.MechDesign.Helpers as HM

from rm_python_lib.MechDesign.Units.Units import m_, mm_, kg_, s_, N_, rpm_, W_
import rm_python_lib.MechDesign.Units.UnitMethods as UM

import rm_python_lib.MechDesign.RnM as RnM
belt = RnM.Belt()
gears = RnM.GearDesign()
chain = RnM.Chain()
B1 = RnM.Shaft()
B2 = RnM.Shaft()
connections = RnM.ShaftConnection()

## Sesion 1



### P T n Calculations
First lets insert known data to the variables. The belt efficiency is taken from the chapter 16 that explains the flat belts (the ones we will be using). Gear efficiency is assumed out of nowhere.

In [13]:
# Given values
belt.n_1 = 1500*rpm_  # Speed of motor
chain.v = 15/3.6*m_/s_  # Speed of sprocket 1
chain.d_1 = UM.All_to_SI(450*mm_)  # Diameter of sprocket 1
chain.P_2 = 4.4*1000*W_  # Power needed at sprocket 1
gears.beta_1 =0    # teeth are straight cut 
gears.beta_2 = 0    # teeth are straight cut 
gears.nu_t = 0.9 # gear transmission efficiency
belt.eta = 0.98 # belt drive efficiency


Having in mind the given data now it is possible to calculate the parameters for the shaft $B2$. Also it is important to mention that the shafts are numbered from left to right. Meaning that motor shaft data is marked with a "$_1$", $B1$ shaft with and index "$_2$" and the $B2$ shaft with and index "$_3$". Since now we assume ideal conditions to compute aproximate power, all three shafts will have the same power because of the conservation of energy.

In [14]:
chain.T_1 = chain.P_2/(2*chain.v/chain.d_1)  # Torque on sprocket 1

w_1 = 2*chain.v/chain.d_1             # Angular velocity of sprocket 1
chain.n_1 = UM.RemoveUnits(w_1/(2*sp.pi)*60*rpm_)*rpm_  # Rotational speed of sprocket 1

gears.P_1 = chain.P_2/ gears.nu_t  # Power needed at gear 1
belt.P = gears.P_1 / belt.eta  # Power needed at pulley 1


t=HM.EqPrint('P_1', belt.P)
t=HM.EqPrint('P_2', gears.P_1)
t=HM.EqPrint('P_3', chain.P_2)
t=HM.EqPrint('T_3', chain.T_1.evalf())
t=HM.EqPrint('n_3', chain.n_1.evalf())

Eq(P_1, 4989.0*W_)

Eq(P_2, 4889.0*W_)

Eq(P_3, 4400.0*W_)

Eq(T_3, 237.6*W_*s_)

Eq(n_3, 176.8*rpm_)

In [15]:
import math
i_tot = chain.n_1/belt.n_1
t=HM.EqPrint('i_tot',i_tot)


belt.n_2 = math.sqrt(UM.RemoveUnits(3*chain.n_1*belt.n_1/5))
t = HM.EqPrint('n_2',belt.n_2*rpm_) 

belt.i = UM.RemoveUnits(belt.n_2/belt.n_1)
t = HM.EqPrint('i_r',belt.i)

gears.omega_1 = UM.rpm_to_rad_s(belt.n_2*rpm_)
gears.omega_2 = UM.rpm_to_rad_s(chain.n_1)

gears.i = gears.omega_2/gears.omega_1
t = HM.EqPrint('i_twk', gears.i)



Eq(i_tot, 0.1179)

Eq(n_2, 398.9*rpm_)

Eq(i_r, 0.266)

Eq(i_twk, 0.4433)

Since now all the rotational speeds are known the torques can be computed.

In [16]:
belt.T_1 = belt.P/ UM.rpm_to_rad_s(belt.n_1)

belt.T_2 = gears.P_1/ gears.omega_1
t = HM.EqPrint('T_1', belt.T_1)
t = HM.EqPrint('T_2', belt.T_2)

Eq(T_1, 31.76*W_*s_)

Eq(T_2, 117.0*W_*s_)

### Motor selection

### Finding the right Ka

### Key calculations
In this section, Ka and the efficiency of the motor is calculated based on some design assumptions.
------------

![Table_3_5b](Table_3_5b.png)

Assuming a medium-duty induction motor (moderate start/stop frequency), continuous full-load operation with occasional mild shocks, a drivetrain susceptible to gear pitting, and 20 hours/day operation, the preliminary estimate for the control/amplifier constant is 𝐾𝐴=1.9. In this context 𝐾𝐴 represents the gain between the control command and stator voltage (i.e. Vs=KA⋅Vcontrol).A higher KA increases available stator voltage and therefore torque for a given controller output, but it also increases stator currents and associated copper and iron losses (losses scale nonlinearly with current). The operating conditions (frequent moderate starts, shocks and gear pitting) will reduce overall mechanical efficiency and increase vibration-related losses and bearing wear, so the effective system efficiency will be lower than nameplate motor efficiency. This estimated 𝐾𝐴 should therefore be validated experimentally and may need to be derated to limit thermal stress and prolong gearbox and bearing life.